In [7]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [4]:
training_df = pd.read_csv("train.csv")

In [8]:
training_df.shape

(800, 248)

In [9]:
training_df.set_index("Timestamp", inplace=True)

In [11]:
training_df.head()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Country,Afghanistan,Albania,Algeria,American Samoa,Andorra,...,Zambia,Zimbabwe,afternoon,evening,morning,night,Ad Topic Line,Ad Line Length,Ad Line Word Length,Clicked on Ad
Timestamp,,,,,,,,,,,,,,,,,,,,,
2016-01-29 00:45:19,-1.003266,-0.475928,-1.074513,-1.369008,Fiji,0,0,0,0,0,...,0,0,0,0,0,1,Optimized intermediate help-desk,-0.247760,-0.451417,1
2016-01-05 20:58:42,0.838468,0.211953,0.176047,1.637189,Liberia,0,0,0,0,0,...,0,0,0,1,0,0,Networked regional Local Area Network,0.629266,3.723539,0
2016-06-18 16:02:34,-1.895712,0.441247,-0.705899,-0.666237,Mongolia,0,0,0,0,0,...,0,0,1,0,0,0,Fully-configurable context-sensitive Graphic I...,3.611156,1.636061,1
2016-06-21 00:52:47,0.386877,-0.705222,1.065619,1.014610,France,0,0,0,0,0,...,0,0,0,0,0,1,Assimilated stable encryption,-0.773976,-0.451417,0
2016-04-17 05:08:52,1.199110,1.358422,-0.643565,-1.009855,Kyrgyz Republic,0,0,0,0,0,...,0,0,0,0,1,0,Inverse discrete extranet,-1.475597,-0.451417,1


In [27]:
training_X_cols = list(training_df.select_dtypes(exclude=["object"]).columns)
training_X_cols.remove('Clicked on Ad')
training_y_cols = 'Clicked on Ad'

In [58]:
param_grid = [
    {"C":[1,10,100],"kernel":["linear","poly","rbf"],"degree":[3,6,9]},
]

In [59]:
grid_search = GridSearchCV(SVC(),param_grid,cv=5, scoring=["neg_mean_squared_error","accuracy"],refit="neg_mean_squared_error")

In [60]:
grid_search.fit(training_df[training_X_cols],training_df[training_y_cols])

GridSearchCV(cv=5, estimator=SVC(),
             param_grid=[{'C': [1, 10, 100], 'degree': [3, 6, 9],
                          'kernel': ['linear', 'poly', 'rbf']}],
             refit='neg_mean_squared_error',
             scoring=['neg_mean_squared_error', 'accuracy'])

In [64]:
best_estimator = grid_search.best_estimator_
print(best_estimator)

SVC(C=10, kernel='linear')


In [62]:
results = grid_search.cv_results_
# print(results.keys.fetchall())
for key in results.keys():
    print(key)
    print(results[key])
    print("\n\n")

mean_fit_time
[0.02255449 0.03771663 0.0296968  0.02094421 0.08317752 0.02931366
 0.02253456 0.08239732 0.03391852 0.02353358 0.03469658 0.0313231
 0.02215071 0.09692416 0.03151526 0.02253962 0.09216137 0.03131838
 0.03389444 0.03429294 0.0331109  0.03392334 0.0947329  0.0315166
 0.03470712 0.09532776 0.03191433]



std_fit_time
[0.00149715 0.00117436 0.0003928  0.00140733 0.00638901 0.00049658
 0.00239215 0.00050621 0.003096   0.00239996 0.00073719 0.00049987
 0.00203583 0.00288836 0.0007977  0.00205384 0.00224108 0.00048661
 0.01051067 0.00080007 0.00159624 0.0085348  0.0012503  0.00079896
 0.00994136 0.00172552 0.00063105]



mean_score_time
[0.00596938 0.01077771 0.00820713 0.00576963 0.02113519 0.00798612
 0.00579605 0.02114682 0.0091558  0.00517535 0.00997715 0.00817952
 0.00497842 0.02033901 0.00797873 0.00538583 0.0201467  0.00857453
 0.00480781 0.00958419 0.00937529 0.00497823 0.01994386 0.00857654
 0.00538568 0.01895733 0.00877686]



std_score_time
[2.78407573e-05 4.05563164

In [63]:
for mean_squared_error, accuracy, params in zip(results["mean_test_neg_mean_squared_error"], results["mean_test_accuracy"], results["params"]):
    print(f"params: {params} mse score: {-mean_squared_error} rmse: {np.sqrt(-mean_squared_error)} accuracy: {accuracy} ")
    print("\n")

params: {'C': 1, 'degree': 3, 'kernel': 'linear'} mse score: 0.0375 rmse: 0.19364916731037085 accuracy: 0.9625 


params: {'C': 1, 'degree': 3, 'kernel': 'poly'} mse score: 0.045 rmse: 0.21213203435596426 accuracy: 0.9550000000000001 


params: {'C': 1, 'degree': 3, 'kernel': 'rbf'} mse score: 0.04125 rmse: 0.203100960115899 accuracy: 0.95875 


params: {'C': 1, 'degree': 6, 'kernel': 'linear'} mse score: 0.0375 rmse: 0.19364916731037085 accuracy: 0.9625 


params: {'C': 1, 'degree': 6, 'kernel': 'poly'} mse score: 0.17750000000000002 rmse: 0.42130748865881795 accuracy: 0.8225000000000001 


params: {'C': 1, 'degree': 6, 'kernel': 'rbf'} mse score: 0.04125 rmse: 0.203100960115899 accuracy: 0.95875 


params: {'C': 1, 'degree': 9, 'kernel': 'linear'} mse score: 0.0375 rmse: 0.19364916731037085 accuracy: 0.9625 


params: {'C': 1, 'degree': 9, 'kernel': 'poly'} mse score: 0.28625 rmse: 0.5350233639758174 accuracy: 0.71375 


params: {'C': 1, 'degree': 9, 'kernel': 'rbf'} mse score: 0.041